In [1]:
!pip install gensim

In [2]:
import gensim.downloader as api

In [3]:
wv = api.load('word2vec-google-news-300')

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [4]:
wv.similarity("great","good")

0.729151

In [5]:
import pandas as pd

In [7]:
df = pd.read_csv('fake_and_real_news.csv')
df.head()

,Text,label
0,Top Trump Surrogate BRUTALLY Stabs Him In The...,Fake
1,U.S. conservative leader optimistic of common ...,Real
2,"Trump proposes U.S. tax overhaul, stirs concer...",Real
3,Court Forces Ohio To Allow Millions Of Illega...,Fake
4,Democrats say Trump agrees to work on immigrat...,Real


In [8]:
df.label.value_counts()

Fake    5000
Real    4900
Name: label, dtype: int64

In [9]:
df['new_label'] = df['label'].map({'Fake':1,'Real':0})

In [10]:
df.head()

,Text,label,new_label
0,Top Trump Surrogate BRUTALLY Stabs Him In The...,Fake,1
1,U.S. conservative leader optimistic of common ...,Real,0
2,"Trump proposes U.S. tax overhaul, stirs concer...",Real,0
3,Court Forces Ohio To Allow Millions Of Illega...,Fake,1
4,Democrats say Trump agrees to work on immigrat...,Real,0


In [11]:
!python -m spacy download en_core_web_lg

2023-11-11 04:18:28.069539: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-11 04:18:28.069644: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-11 04:18:28.069760: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-11 04:18:28.091002: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-11 04:18:29.470891: W tensorflow/compiler/

In [28]:
import spacy
nlp = spacy.load('en_core_web_lg')
def preprocess_and_vectorize(text):
  doc = nlp(text)
  f_token = []
  for token in doc:
    if not (token.is_punct and token.is_stop):
      f_token.append(token.lemma_)
  return wv.get_mean_vector(" ".join(f_token))


In [20]:
preprocess_and_vectorize("Hey dont worry for that buddy ")

'hey do not worry for that buddy'

In [25]:
import numpy as np
v1 = wv['worry']
v2 = wv['understand']
np.mean([v1,v2],axis = 0)[:4]

array([ 0.00976562, -0.00561523, -0.08905029,  0.01330566], dtype=float32)

In [27]:
wv.get_mean_vector(['worry','understand'],pre_normalize=False)[:4]

array([ 0.00976562, -0.00561523, -0.08905029,  0.01330566], dtype=float32)

In [ ]:
df['vector'] = df['Text'].apply(lambda x: preprocess_and_vectorize(x))

In [ ]:
df.head()

In [ ]:
from sklearn.model_selection import train_test_split


#Do the 'train-test' splitting with test size of 20% with random state of 2022 and stratify sampling too
X_train, X_test, y_train, y_test = train_test_split(
    df.vector.values,
    df.label_num,
    test_size=0.2, # 20% samples will go to test dataset
    random_state=2022,
    stratify=df.label_num
)

In [ ]:
X_train_2d = np.stack(X_train)
X_test_2d =  np.stack(X_test)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report

#1. creating a GradientBoosting model object
clf = GradientBoostingClassifier()

#2. fit with all_train_embeddings and y_train
clf.fit(X_train_2d, y_train)


#3. get the predictions for all_test_embeddings and store it in y_pred
y_pred = clf.predict(X_test_2d)


#4. print the classfication report
print(classification_report(y_test, y_pred))